# Regression of Vote Characteristics on CAR

In [1]:
import stata_setup

stata_setup.config('/Applications/StataNow', 'mp')


  ___  ____  ____  ____  ____ ®
 /__    /   ____/   /   ____/      StataNow 19.5
___/   /   /___/   /   /___/       MP—Parallel Edition

 Statistics and Data Science       Copyright 1985-2025 StataCorp LLC
                                   StataCorp
                                   4905 Lakeway Drive
                                   College Station, Texas 77845 USA
                                   800-782-8272        https://www.stata.com
                                   979-696-4600        service@stata.com

Stata license: Unlimited-user 2-core network, expiring  9 Sep 2026
Serial number: 501909305069
  Licensed to: 罗奕辰
               UCL

Notes:
      1. Unicode is supported; see help unicode_advice.
      2. More than 2 billion observations are allowed; see help obs_advice.
      3. Maximum number of variables is set to 5,000 but can be increased;
          see help set_maxvar.


In [2]:
%%stata

clear all
set more off
set varabbrev off
version 18

* Paths
global PROCESSED_DATA "processed_data"
global TABLES "tables"


. 
. clear all

. set more off

. set varabbrev off

. version 18

. 
. * Paths
. global PROCESSED_DATA "processed_data"

. global TABLES "tables"

. 


In [3]:
%%stata
local stage created
import delimited using "$PROCESSED_DATA/event_study_panel_`stage'.csv", clear

* Parse date
gen day = date(date, "YMD")
format day %td

* Create year variable
gen year = year(day)

* Create quarter variable
gen quarter = qofd(day)
format quarter %tq

* Ensure index is numeric (safeguard if it came in as string)
capture confirm numeric variable index
if _rc {
    destring index, replace ignore("[]")
}

* Encode categorical variables
encode gecko_id, gen(token)
encode space,    gen(dao)

egen token_year = group(token year)
egen token_quarter = group(token quarter)

* Define windows and specs
local window 5
local vars support professionalism unanimity objectiveness
eststo clear

keep if index == `window'

* VIF test
qui reg car `vars'
estat vif

* Correlation matrix
pwcorr `vars', sig star(0.05)

* Run regression
reghdfe car `vars', absorb(year dao) vce(cluster dao)
eststo car

* Tag FE rows (Y/N) for this spec
estadd local fe_token "Y"
estadd local fe_time  "Y"


. local stage created

. import delimited using "$PROCESSED_DATA/event_study_panel_`stage'.csv", clear
(encoding automatically selected: ISO-8859-1)
(36 vars, 17,435 obs)

. 
. * Parse date
. gen day = date(date, "YMD")

. format day %td

. 
. * Create year variable
. gen year = year(day)

. 
. * Create quarter variable
. gen quarter = qofd(day)

. format quarter %tq

. 
. * Ensure index is numeric (safeguard if it came in as string)
. capture confirm numeric variable index

. if _rc {
.     destring index, replace ignore("[]")
. }

. 
. * Encode categorical variables
. encode gecko_id, gen(token)

. encode space,    gen(dao)

. 
. egen token_year = group(token year)

. egen token_quarter = group(token quarter)

. 
. * Define windows and specs
. local window 5

. local vars support professionalism unanimity objectiveness

. eststo clear

. 
. keep if index == `window'
(15,850 observations deleted)

. 
. * VIF test
. qui reg car `vars'

. estat vif

    Variable |       VIF       1/VIF

## Participation

In [4]:
%%stata

import delimited using "$PROCESSED_DATA/proposals_panel.csv", clear

* Parse date
gen day = date(date, "YMD")
format day %td
gen month = month(day)
gen quarter = quarter(day)

gen year = year(day)
encode gecko_id, gen(token)
encode space,    gen(dao)

* Dependent variables
local dep non_whale_participation whale_participation

foreach d of local dep {


    * Run regression
    reghdfe `d' `vars', absorb(year dao) vce(cluster token)
    eststo `d'

    * Tag FE rows (Y/N) for this spec
    estadd local fe_token "Y"
    estadd local fe_time  "Y"
}

* Label variables
label var support          "\${\it Support}_{i,t}\$"
label var professionalism  "\${\it Professionalism}_{i,t}\$"
label var unanimity        "\${\it Unanimity}_{i,t}\$"
label var objectiveness    "\${\it Objectiveness}_{i,t}\$"
label var hhi_post_number  "\${\it HHI}_{i,t}^{\# of Post}\$"

* Export a single big LaTeX table
esttab                                                         ///
    non_whale_participation whale_participation car            ///
    using "$TABLES/reg_discussion.tex", replace    ///
    se star(* 0.10 ** 0.05 *** 0.01) label nogaps nocon        ///
    nonotes booktabs nomtitles                                 ///
    b(%9.3f) se(%9.2f)                                         ///
    mgroups("\${\it Participation}^{Small}_{i,t}\$"            ///
            "\${\it Participation}^{Whale}_{i,t}\$"            /// 
            "CAR[-5,5]",                                       ///
            span                                               ///
            pattern(1 1 1)                                     ///
            prefix(\multicolumn{@span}{c}{)                    ///
            suffix(})                                          ///
            erepeat(\cmidrule(lr){@span}) )                    ///
    substitute("\_" "_")                                       ///
    stats(fe_token fe_time N r2_a,                             ///      
            fmt(0 0 %9.0fc %9.3f) labels("Token FE" "Year FE" "Observations" "Adjusted R²"))

    


. 
. import delimited using "$PROCESSED_DATA/proposals_panel.csv", clear
(encoding automatically selected: ISO-8859-1)
(51 vars, 2,841 obs)

. 
. * Parse date
. gen day = date(date, "YMD")

. format day %td

. gen month = month(day)

. gen quarter = quarter(day)

. 
. gen year = year(day)

. encode gecko_id, gen(token)

. encode space,    gen(dao)

. 
. * Dependent variables
. local dep non_whale_participation whale_participation

. 
. foreach d of local dep {
  2. 
. 
.     * Run regression
.     reghdfe `d' `vars', absorb(year dao) vce(cluster token)
  3.     eststo `d'
  4. 
.     * Tag FE rows (Y/N) for this spec
.     estadd local fe_token "Y"
  5.     estadd local fe_time  "Y"
  6. }
(dropped 1 singleton observations)
(MWFE estimator converged in 7 iterations)

HDFE Linear regression                            Number of obs   =        847
Absorbing 2 HDFE groups                           F(   4,     18) =       0.78
Statistics robust to heteroskedasticity           Prob > F     